In [ ]:
import numpy as np
from packaging import version
import subprocess
import sys
# Following code required owing to the issue here https://github.com/h2oai/h2o-3/issues/16462 , once numpy 2 is supported the code can be removed
# Check if current NumPy is >= 2
if version.parse(np.__version__) >= version.parse("2.0"):
    print(f"Detected NumPy {np.__version__}, reinstalling < 2.0...")
    !pip install --force-reinstall --no-cache-dir "numpy<2"
    # Restart runtime manually
else:
    print(f"NumPy {np.__version__} is already < 2.0. No action needed.")

NumPy 1.26.4 is already < 2.0. No action needed.


In [ ]:
!pip install h2o
import os

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.9/265.9 MB 3.9 MB/s eta 0:00:00


In [ ]:
import re

In [ ]:
import h2o
import pandas as pd
# helper function to format grid search results

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
h2o.init(max_mem_size = "45g")


Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.28" 2025-07-15; OpenJDK Runtime Environment (build 11.0.28+6-post-Ubuntu-1ubuntu122.04.1); OpenJDK 64-Bit Server VM (build 11.0.28+6-post-Ubuntu-1ubuntu122.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.11/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpdwgk8di1
  JVM stdout: /tmp/tmpdwgk8di1/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpdwgk8di1/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.7
H2O_cluster_version_age:,4 months and 18 days
H2O_cluster_name:,H2O_from_python_unknownUser_m8h3fq
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,45 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [ ]:
cases2010to2018_h2o = h2o.import_file('/content/drive/MyDrive/Law/Model2abbaselineData_2010to2018_merged_criminal_regr.csv')
#cases2010to2018_h2o = h2o.import_file('/content/drive/MyDrive/Law/automl_2010to2018_merged_all.csv')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
cases2010to2018_h2o = cases2010to2018_h2o[((cases2010to2018_h2o['pendency_decision_filing'] <= 100000))]

In [ ]:
cases2010to2018_h2o.types

{'state_code': 'int',
 'female_defendant': 'int',
 'female_petitioner': 'enum',
 'female_adv_def': 'int',
 'female_adv_pet': 'int',
 'billable_ipc': 'enum',
 'female_judge_filing': 'enum',
 'female_judge_decision': 'enum',
 'judgediff': 'int',
 'pendency_decision_filing': 'int',
 'pendency_first_filing': 'int',
 'muslim_defendant': 'enum',
 'muslim_petitioner': 'enum',
 'muslim_adv_def': 'int',
 'muslim_adv_pet': 'int',
 'court_details': 'enum',
 'state_district': 'enum',
 'pendency_model_multiyear': 'enum',
 'filing_judge_tenure_at_filing': 'int',
 'decision_judge_tenure_at_decision': 'int',
 'act_s': 'enum',
 'section_s': 'int',
 'type_name_s': 'enum',
 'judge_position_s': 'enum',
 'number_sections_ipc_category': 'enum'}

In [ ]:
#Remove all columns not available at time of case filing

# List of columns to keep
columns_to_keep = cases2010to2018_h2o.columns

columns_to_keep.remove("female_judge_decision")
columns_to_keep.remove("decision_judge_tenure_at_decision")
columns_to_keep.remove("pendency_first_filing")
columns_to_keep.remove("judgediff")
#Drop columns related to advocate and religion
columns_to_keep.remove("muslim_defendant")
columns_to_keep.remove("muslim_petitioner")
columns_to_keep.remove("muslim_adv_def")
columns_to_keep.remove("muslim_adv_pet")
columns_to_keep.remove("female_adv_def")
columns_to_keep.remove("female_adv_pet")
columns_to_keep.remove("female_defendant")
columns_to_keep.remove("female_petitioner")
columns_to_keep.remove("female_judge_filing")


# Use the assign method to drop columns
cases2010to2018_h2o = cases2010to2018_h2o[columns_to_keep]



In [ ]:
cases2010to2018_h2o.types

{'state_code': 'int',
 'billable_ipc': 'enum',
 'pendency_decision_filing': 'int',
 'court_details': 'enum',
 'state_district': 'enum',
 'pendency_model_multiyear': 'enum',
 'filing_judge_tenure_at_filing': 'int',
 'act_s': 'enum',
 'section_s': 'int',
 'type_name_s': 'enum',
 'judge_position_s': 'enum',
 'number_sections_ipc_category': 'enum'}

In [ ]:
cases2010to2018_h2o

state_code,billable_ipc,pendency_decision_filing,court_details,state_district,pendency_model_multiyear,filing_judge_tenure_at_filing,act_s,section_s,type_name_s,judge_position_s,number_sections_ipc_category
25,NA,3,25_2_5,25_2,ontime,2,Code of Criminal Procedure,104,NA,Judicial Magistrate Court,NA
13,NA,0,13_25_3,13_25,ontime,49,Code of Criminal Procedure,104,transfer application (criminal),Chief Judicial Magistrate,NA
17,non-bailable,0,17_31_2,17_31,ontime,26,The Indian Penal Code,130,NA,Civil Court,5 to 10
23,non-bailable,14,23_15_1,23_15,ontime,9,The Indian Penal Code,130,NA,District And Sessions Court,5 to 10
2,non-bailable,2,2_7_17,2_7,ontime,3,The Indian Penal Code,134,cc,Civil Judge Junior Division,5 to 10
3,non-bailable,5,3_18_11,3_18,ontime,12,The Indian Penal Code,134,c.c.,principal civil judge,four
4,non-bailable,31,4_2_5,4_2,delayed,12,The Indian Penal Code,134,cc,Chief Judicial Magistrate,three
3,non-bailable,1,3_25_5,3_25,ontime,17,The Indian Penal Code,134,NA,principal civil judge,three
3,non-bailable,1,3_20_1,3_20,ontime,19,The Indian Penal Code,134,a.i.r.misc,small cause court,one
3,non-bailable,4,3_20_1,3_20,ontime,4,The Indian Penal Code,134,a.i.r.misc,small cause court,one


In [ ]:
cases2010to2018_h2o['section_s'] = cases2010to2018_h2o['section_s'].asfactor()
cases2010to2018_h2o['state_code'] = cases2010to2018_h2o['state_code'].asfactor()
cases2010to2018_h2o['bailable_ipc'] = cases2010to2018_h2o['billable_ipc']


In [ ]:
cases2010to2018_h2o.types

{'state_code': 'enum',
 'billable_ipc': 'enum',
 'pendency_decision_filing': 'int',
 'court_details': 'enum',
 'state_district': 'enum',
 'pendency_model_multiyear': 'enum',
 'filing_judge_tenure_at_filing': 'int',
 'act_s': 'enum',
 'section_s': 'enum',
 'type_name_s': 'enum',
 'judge_position_s': 'enum',
 'number_sections_ipc_category': 'enum',
 'bailable_ipc': 'enum'}

In [ ]:
cases2010to2018_h2o.head()

state_code,billable_ipc,pendency_decision_filing,court_details,state_district,pendency_model_multiyear,filing_judge_tenure_at_filing,act_s,section_s,type_name_s,judge_position_s,number_sections_ipc_category,bailable_ipc
25,NA,3,25_2_5,25_2,ontime,2,Code of Criminal Procedure,104,NA,Judicial Magistrate Court,NA,NA
13,NA,0,13_25_3,13_25,ontime,49,Code of Criminal Procedure,104,transfer application (criminal),Chief Judicial Magistrate,NA,NA
17,non-bailable,0,17_31_2,17_31,ontime,26,The Indian Penal Code,130,NA,Civil Court,5 to 10,non-bailable
23,non-bailable,14,23_15_1,23_15,ontime,9,The Indian Penal Code,130,NA,District And Sessions Court,5 to 10,non-bailable
2,non-bailable,2,2_7_17,2_7,ontime,3,The Indian Penal Code,134,cc,Civil Judge Junior Division,5 to 10,non-bailable
3,non-bailable,5,3_18_11,3_18,ontime,12,The Indian Penal Code,134,c.c.,principal civil judge,four,non-bailable
4,non-bailable,31,4_2_5,4_2,delayed,12,The Indian Penal Code,134,cc,Chief Judicial Magistrate,three,non-bailable
3,non-bailable,1,3_25_5,3_25,ontime,17,The Indian Penal Code,134,NA,principal civil judge,three,non-bailable
3,non-bailable,1,3_20_1,3_20,ontime,19,The Indian Penal Code,134,a.i.r.misc,small cause court,one,non-bailable
3,non-bailable,4,3_20_1,3_20,ontime,4,The Indian Penal Code,134,a.i.r.misc,small cause court,one,non-bailable


In [ ]:
print(len(cases2010to2018_h2o))

16482391


In [ ]:
cases2010to2018_train, cases2010to2018_test, cases2010to2018_valid = cases2010to2018_h2o.split_frame(ratios=[.7, .15])

### Load Model1c

In [ ]:
model1c_path = "/content/drive/MyDrive/Colab Notebooks/Law/paper2&3/paperpredictivecriminal/ToShare/Results/ResultsModel1c/saved_models/drf_grid_model_1.zip"
model1c = h2o.import_mojo(model1c_path)


generic Model Build progress: |██████████████████████████████████████████████████| (done) 100%


In [ ]:
### Load Model 2a
model2a_path = "/content/drive/MyDrive/Colab Notebooks/Law/paper2&3/paperpredictivecriminal/ToShare/Results/ResultsModel2a/saved_models/DRF_1_AutoML_1_20250808_152030.zip"
model2a = h2o.import_mojo(model2a_path)

generic Model Build progress: |██████████████████████████████████████████████████| (done) 100%


In [ ]:
### Load Model 2b
model2b_path = "/content/drive/MyDrive/Colab Notebooks/Law/paper2&3/paperpredictivecriminal/ToShare/Results/ResultsModel2b/saved_models/GBM_1_AutoML_1_20250808_152037.zip"
model2b = h2o.import_mojo(model2b_path)

generic Model Build progress: |██████████████████████████████████████████████████| (done) 100%


In [ ]:
### Load Model 2 Baseline
model2_baseline = "/content/drive/MyDrive/Colab Notebooks/Law/paper2&3/paperpredictivecriminal/ToShare/Results/ResultsModel2Baseline/saved_models/DRF_1_AutoML_1_20250808_151249.zip"
model2_baseline = h2o.import_mojo(model2_baseline)

generic Model Build progress: |██████████████████████████████████████████████████| (done) 100%


In [ ]:
import os
import random
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score
from scipy.stats import ttest_rel
import warnings
warnings.filterwarnings('ignore')

# Parameters
n_runs = 100
sample_size = 1000 # adjust if needed
threshold_months = 24  # routing threshold
prob_threshold = [0.85]

# Store results
results = []
for prob in prob_threshold:
  for run in range(n_runs):
      # Random sample indices — then sort them for H2O slicing
      random_indices = sorted(random.sample(range(cases2010to2018_test.nrow), sample_size))
      test_sample = cases2010to2018_test[random_indices, :]

      # Ground truth duration and delay status
      y_true_duration = test_sample['pendency_decision_filing'].as_data_frame().iloc[:, 0].values
      y_true_delay = test_sample['pendency_model_multiyear'].as_data_frame().iloc[:, 0].apply(
          lambda x: 0 if x == "delayed" else 1
      ).values

       # ---- Approach 0: True Delay classification for routing to Model-2baseline ----
      routed_preds_td = []
      for i, pred_class in enumerate(y_true_delay):
          if pred_class == 1:  # on-time
              routed_preds_td.append(model2_baseline.predict(test_sample[i, :]).as_data_frame().iloc[0, 0])
          else:  # delayed
              routed_preds_td.append(model2_baseline.predict(test_sample[i, :]).as_data_frame().iloc[0, 0])
      routed_preds_td = np.array(routed_preds_td)

      rmse_0 = np.sqrt(mean_squared_error(y_true_duration, routed_preds_td))
      mae_0 = mean_absolute_error(y_true_duration, routed_preds_td)
      acc_class_0 = accuracy_score(y_true_delay, y_true_delay)
      align_0 = np.mean((y_true_delay) == (routed_preds_td <= threshold_months))

      # ---- Approach 1: True Delay classification for routing to Model-2a or 2b ----
      routed_preds_td = []
      for i, pred_class in enumerate(y_true_delay):
          if pred_class == 1:  # on-time
              routed_preds_td.append(model2a.predict(test_sample[i, :]).as_data_frame().iloc[0, 0])
          else:  # delayed
              routed_preds_td.append(model2b.predict(test_sample[i, :]).as_data_frame().iloc[0, 0])
      routed_preds_td = np.array(routed_preds_td)

      rmse_1 = np.sqrt(mean_squared_error(y_true_duration, routed_preds_td))
      mae_1 = mean_absolute_error(y_true_duration, routed_preds_td)
      acc_class_1 = accuracy_score(y_true_delay, y_true_delay)
      align_1 = np.mean((y_true_delay) == (routed_preds_td <= threshold_months))


      # ---- Approach 2: Model-2 baseline regression for routing to Model-2a or 2b ----
      y_pred_dur_baseline = model2_baseline.predict(test_sample).as_data_frame().iloc[:, 0].values

      y_pred_dur_baseline_routing = y_pred_dur_baseline
      routed_preds = []
      for i, pred_dur in enumerate(y_pred_dur_baseline_routing):
          if pred_dur <= threshold_months:
              routed_preds.append(model2a.predict(test_sample[i, :]).as_data_frame().iloc[0, 0])
          else:
              routed_preds.append(model2b.predict(test_sample[i, :]).as_data_frame().iloc[0, 0])
      routed_preds = np.array(routed_preds)

      rmse_2 = np.sqrt(mean_squared_error(y_true_duration, routed_preds))
      mae_2 = mean_absolute_error(y_true_duration, routed_preds)
      acc_class_2 = accuracy_score(y_true_delay, (y_pred_dur_baseline_routing
                                                  <=threshold_months).astype(int))
      align_2 = np.mean((y_pred_dur_baseline_routing > threshold_months) == (routed_preds > threshold_months))

      # ---- Approach 3: Model-1c classification for routing to Model-2a or 2b ----
      y_pred_class_1c = model1c.predict(test_sample).as_data_frame()['predict'].apply(
          lambda x: 0 if x == 'delayed' else 1
      ).values
      routed_preds_1c = []
      for i, pred_class in enumerate(y_pred_class_1c):
          if pred_class == 1:  # on-time
              routed_preds_1c.append(model2a.predict(test_sample[i, :]).as_data_frame().iloc[0, 0])
          else:  # delayed
              routed_preds_1c.append(model2b.predict(test_sample[i, :]).as_data_frame().iloc[0, 0])
      routed_preds_1c = np.array(routed_preds_1c)

      rmse_3 = np.sqrt(mean_squared_error(y_true_duration, routed_preds_1c))
      mae_3 = mean_absolute_error(y_true_duration, routed_preds_1c)
      acc_class_3 = accuracy_score(y_true_delay, y_pred_class_1c)
      align_3 = np.mean(y_pred_class_1c == (routed_preds_1c <= threshold_months))


      # ---- Approach 4: Model-1c + baseline classification for routing to Model-2a or 2b or baseline ----
      routed_preds_mix = []
      y_pred_class_mix = []

      for i, (pred_class, pred_dur) in enumerate(zip(y_pred_class_1c, y_pred_dur_baseline_routing)):
          #print(f"Actual Delay {y_true_delay[i]}")
          #print(f"Actual Duration {y_true_duration[i]}")
          #print(f"Predicted Delay 1c {pred_class}")
          #print(f"Predicted Duration 2 Baseline {pred_dur}")

          if pred_class == 1:  # on-time
              #print(f"Predicted Duration 2a {model2a.predict(test_sample[i, :]).as_data_frame().iloc[0, 0]}")
              if (pred_dur <= threshold_months) :
                routed_preds_mix.append(model2a.predict(test_sample[i, :]).as_data_frame().iloc[0, 0])
                y_pred_class_mix.append(1)
                #print(f"Predicted Model 2a added without probability {model2a.predict(test_sample[i, :]).as_data_frame().iloc[0, 0]}")
              else:
                model_prediction_df = model1c.predict(test_sample[i, :]).as_data_frame()
                label = model_prediction_df['predict'].iloc[0]
                # Access probability based on the predicted label
                probability = model_prediction_df[label].iloc[0]
                #print(f"label {label}")
                #print(f"probability {probability}")
                if probability > prob:
                    routed_preds_mix.append(model2a.predict(test_sample[i, :]).as_data_frame().iloc[0, 0])
                    y_pred_class_mix.append(1)
                    #print(f"Predicted Model 2a probability mode added {model2a.predict(test_sample[i, :]).as_data_frame().iloc[0, 0]}")
                else:
                  routed_preds_mix.append(model2_baseline.predict(test_sample[i, :]).as_data_frame().iloc[0, 0])
                  #print(f"Predicted Model 2 Baseline probability mode added {model2_baseline.predict(test_sample[i, :]).as_data_frame().iloc[0, 0]}")

                  if model2_baseline.predict(test_sample[i, :]).as_data_frame().iloc[0, 0] <= threshold_months :
                      y_pred_class_mix.append(1)
                  else:
                      y_pred_class_mix.append(0)
          else:  # delayed
              #print(f"Predicted Duration 2b {model2b.predict(test_sample[i, :]).as_data_frame().iloc[0, 0]}")
              if (pred_dur > threshold_months) :
                routed_preds_mix.append(model2b.predict(test_sample[i, :]).as_data_frame().iloc[0, 0])
                y_pred_class_mix.append(0)
                #print(f"Predicted Model 2b added without probability {model2b.predict(test_sample[i, :]).as_data_frame().iloc[0, 0]}")
              else:
                model_prediction_df = model1c.predict(test_sample[i, :]).as_data_frame()
                label = model_prediction_df['predict'].iloc[0]
                # Access probability based on the predicted label
                probability = model_prediction_df[label].iloc[0]
                #print(f"label {label}")
                #print(f"probability {probability}")
                if probability > prob:
                    routed_preds_mix.append(model2b.predict(test_sample[i, :]).as_data_frame().iloc[0, 0])
                    y_pred_class_mix.append(0)
                    #print(f"Predicted Model 2b probability mode added {model2b.predict(test_sample[i, :]).as_data_frame().iloc[0, 0]}")

                else:
                  routed_preds_mix.append(model2_baseline.predict(test_sample[i, :]).as_data_frame().iloc[0, 0])
                  #print(f"Predicted Model 2 Baseline probability mode added {model2_baseline.predict(test_sample[i, :]).as_data_frame().iloc[0, 0]}")

                  if model2_baseline.predict(test_sample[i, :]).as_data_frame().iloc[0, 0] <= threshold_months :
                      y_pred_class_mix.append(1)
                  else:
                      y_pred_class_mix.append(0)

      routed_preds_mix = np.array(routed_preds_mix)

      rmse_4 = np.sqrt(mean_squared_error(y_true_duration, routed_preds_mix))
      mae_4 = mean_absolute_error(y_true_duration, routed_preds_mix)
      acc_class_4 = accuracy_score(y_true_delay, y_pred_class_mix)
      align_4 = np.mean(y_pred_class_mix == (routed_preds_mix <= threshold_months))

      results.append({
       # Approach 0: True Delay routing to Model-2 baseline
      'RMSE_TrueDelay+Model2baseline': rmse_0,
      'MAE_TrueDelay+Model2baseline': mae_0,
      'ACC_Class_TrueDelay+Model2baseline': acc_class_0,
      'Align_TrueDelay+Model2baseline': align_0,

      # Approach 1: True Delay routing to Model-2a/b
      'RMSE_TrueDelay+Model2a_b': rmse_1,
      'MAE_TrueDelay+Model2a_b': mae_1,
      'ACC_Class_TrueDelay+Model2a_b': acc_class_1,
      'Align_TrueDelay+Model2a_b': align_1,

      # Approach 2: Model-2 baseline routing to Model-2a/b
      'RMSE_Model2Baseline+Model2a_b': rmse_2,
      'MAE_Model2Baseline+Model2a_b': mae_2,
      'ACC_Class_Model2Baseline+Model2a_b': acc_class_2,
      'Align_Model2Baseline+Model2a_b': align_2,

      # Approach 3: Model-1c classification routing to Model-2a/b
      'RMSE_Model1c+Model2a_b': rmse_3,
      'MAE_Model1c+Model2a_b': mae_3,
      'ACC_Class_Model1c+Model2a_b': acc_class_3,
      'Align_Model1c+Model2a_b': align_3,

      # Approach 4: Model-1c + Model-2 baseline routing to Model-2a/b/baseline
      'RMSE_Model1c+Baseline+Model2a_b': rmse_4,
      'MAE_Model1c+Baseline+Model2a_b': mae_4,
      'ACC_Class_Model1c+Baseline+Model2a_b': acc_class_4,
      'Align_Model1c+Baseline+Model2a_b': align_4
  })


  # Aggregate results
  df_results = pd.DataFrame(results)
  #file_name = f"/content/drive/MyDrive/Colab Notebooks/Law/paper2&3/paperpredictivecriminal/ToShare/ablation_results_{prob*100}.csv"
  file_name = f"/content/drive/MyDrive/Colab Notebooks/Law/paper2&3/paperpredictivecriminal/ToShare/ablation_results_test.csv"

  df_results.to_csv(file_name, index=False)
  summary = df_results.mean().to_frame(name='Mean').join(df_results.std().to_frame(name='Std'))

  print(summary)

generic prediction progress: |███████████████████████████████████████████████████| (done) 100%
generic prediction progress: |███████████████████████████████████████████████████| (done) 100%
generic prediction progress: |███████████████████████████████████████████████████| (done) 100%
generic prediction progress: |███████████████████████████████████████████████████| (done) 100%
generic prediction progress: |███████████████████████████████████████████████████| (done) 100%
generic prediction progress: |███████████████████████████████████████████████████| (done) 100%
generic prediction progress: |███████████████████████████████████████████████████| (done) 100%
generic prediction progress: |███████████████████████████████████████████████████| (done) 100%
generic prediction progress: |███████████████████████████████████████████████████| (done) 100%
generic prediction progress: |███████████████████████████████████████████████████| (done) 100%
generic prediction progress: |████████████████████